In [3]:
import numpy as np
import pandas as pd
import datetime as dt
from range_key_dict import RangeKeyDict
import compass_utils.special_years
starttime = dt.datetime.now()
current_year = int(starttime.year)

In [4]:
reference_start_date = dt.date(1999,8,1)  ### This should be the first day of FY 2001
str(reference_start_date)

'1999-08-01'

In [6]:
###  Note: Cisco's fiscal year is 364 days normally. Every 6 years, their cycle adds an additional week, and their fiscal year becomes 371 days long.
###  The list, "special_years", below, lists the years which were lengthened from 364 days to 371 days.

how_many_years_to_track = 3

reference_year_day_count = 364
special_years_day_count = 371
special_year_startyear = 2004
special_year_endyear = current_year + how_many_years_to_track
special_years = compass_utils.special_years.get_special_years()     #list(range(special_year_startyear, special_year_endyear, 6))
years_list = list(range(2000,current_year + how_many_years_to_track + 1,1))

# special_years = [2004,2010,2016,2022,2028,2034,2040,2046,2052,2058,2064,2070] # these are the years in which the days_in_year_count 
                                                                              # changes from 364 to 371 to reset the fiscal year cycle

In [8]:
##Build list of years, incrementing by 1 starting from 2020

# years_list = list(range(2000,2051,1))

In [9]:
year_day_count = {}
for year in years_list:
    if year in special_years:
        day_count = 371
    else:
        day_count = 364
    year_day_count[year] = day_count

In [10]:
# year_day_count
# dt.date.isoformat(reference_start_date)

In [11]:
dict_dates = {}
# dict_dates[str(reference_start_date)] = 1
day_increment = 1
j = 0

for y in years_list:
    k = 0
    for i in range(1,year_day_count[y]+1):     #this range is a count of days in the fiscal year indicated in the dict "year_day_count"
        k+=1
        dict_dates[str(reference_start_date + dt.timedelta(days=j))] = k
        j+=1

In [12]:
# dict_dates

In [13]:
dict_fiscalyear = {}
# dict_fiscalyear[str(reference_start_date)] = 2020
#day_increment = 1
j = 0

for y in years_list:
#     k = 0
    for i in range(1,year_day_count[y]+1):     #this range is a count of days in the fiscal year indicated in the dict "year_day_count"
        dict_fiscalyear[str(reference_start_date + dt.timedelta(days=j))] = y
        j+=1

In [14]:
dict_fiscalyear['2020-07-26']

2021

In [15]:
daynumbers = dict_dates.values()
fiscalyears = dict_fiscalyear.values()
dates = dict_dates.keys()

df = pd.DataFrame(list(zip(dates,fiscalyears,daynumbers)),columns=['date','fiscalyear','daynumber'])

In [16]:
df

,date,fiscalyear,daynumber
0,1999-08-01,2000,1
1,1999-08-02,2000,2
2,1999-08-03,2000,3
3,1999-08-04,2000,4
4,1999-08-05,2000,5
...,...,...,...
9123,2024-07-23,2024,360
9124,2024-07-24,2024,361
9125,2024-07-25,2024,362
9126,2024-07-26,2024,363


In [17]:
qlist = [] #qlist is short for quarter_list, containing tuples for each permutation of (year, quarter, days in the quarter)
for year in years_list:
    for q in list([1,2,3,4]):
        if year in special_years and q == 3:
            fiscalquarter = (year, q, (14*7))
        else:
            fiscalquarter = (year, q ,(13*7))
        qlist.append(fiscalquarter)

In [18]:
dfq = pd.DataFrame(qlist, columns=['fiscalyear','fiscalquarter','days_in_quarter'])

In [19]:
dfqgb = dfq.groupby(['fiscalyear'])['days_in_quarter'].cumsum()

In [20]:
dfqgb

0      91
1     182
2     273
3     364
4      91
     ... 
95    364
96     91
97    182
98    273
99    364
Name: days_in_quarter, Length: 100, dtype: int64

In [21]:
pdq = pd.merge(dfq,dfqgb, left_on = dfq.index, right_on=dfqgb.index).drop(columns=['key_0']).rename(columns={'days_in_quarter_x':'days_in_quarter','days_in_quarter_y':'cume_days_in_quarter'})

In [22]:
gx = pd.merge(pdq,pdq,how = 'left', left_on = [pdq.fiscalyear,pdq.index], right_on = [pdq.fiscalyear,pdq.index+1]).drop(columns=['key_0','key_1','fiscalyear_y','fiscalquarter_y','days_in_quarter_y']).fillna(0).rename(columns={'fiscalyear_x':'fiscalyear','fiscalquarter_x':'fiscalquarter','days_in_quarter_x':'days_in_quarter','cume_days_in_quarter_x':'quarterdays_end','cume_days_in_quarter_y':'quarterdays_begin'})

In [23]:
test = pd.merge(df,gx, how='left',left_on = [df.fiscalyear], right_on = [gx.fiscalyear])

In [24]:
test = test.loc[(test['daynumber'] > test['quarterdays_begin']) & (test['daynumber'] <= test['quarterdays_end'])].drop(columns=['fiscalyear_y','key_0']).rename(columns={'fiscalyear_x':'fiscalyear'})

In [25]:
# test.reset_index(inplace=True)

In [26]:
test

,date,fiscalyear,daynumber,fiscalquarter,days_in_quarter,quarterdays_end,quarterdays_begin
0,1999-08-01,2000,1,1,91,91,0.0
4,1999-08-02,2000,2,1,91,91,0.0
8,1999-08-03,2000,3,1,91,91,0.0
12,1999-08-04,2000,4,1,91,91,0.0
16,1999-08-05,2000,5,1,91,91,0.0
...,...,...,...,...,...,...,...
36495,2024-07-23,2024,360,4,91,364,273.0
36499,2024-07-24,2024,361,4,91,364,273.0
36503,2024-07-25,2024,362,4,91,364,273.0
36507,2024-07-26,2024,363,4,91,364,273.0


In [27]:
n_year_rkd_monthOfQtr = RangeKeyDict({
    (1, 29): 1,
    (29, 57): 2,
    (57, 92): 3,
    (92, 120): 1,
    (120, 148): 2,
    (148, 183): 3,
    (183, 211): 1,
    (211, 239): 2,
    (239, 274): 3,
    (274, 302): 1,
    (302, 330): 2,
    (330, 365): 3,      
    })

In [28]:
s_year_rkd_monthOfQtr = RangeKeyDict({
    (1, 29): 1,  ### All keys in the 2nd position are 1 greater than the actual day count because the range function is n + 1
    (29, 57): 2,
    (57, 92): 3,
    (92, 120): 1,
    (120, 148): 2,
    (148, 183): 3,
    (183, 218): 1,
    (218, 246): 2,
    (246, 281): 3,
    (281, 309): 1,
    (309, 337): 2,
    (337, 372): 3,      
    })

In [29]:
def get_monthOfQuarter(row):
    if row.fiscalyear in special_years:
        return s_year_rkd_monthOfQtr[row.daynumber]
    else:
        return n_year_rkd_monthOfQtr[row.daynumber]    

In [30]:
test['month_of_quarter'] = test.apply(get_monthOfQuarter, axis=1)

In [31]:
test.loc[(test['fiscalyear']==2022) & (test['daynumber']==211)]

,date,fiscalyear,daynumber,fiscalquarter,days_in_quarter,quarterdays_end,quarterdays_begin,month_of_quarter
32986,2022-02-27,2022,211,3,91,273,182.0,2


In [32]:
n_year_rkd_monthOfYear = RangeKeyDict({
    (1, 29): 1,
    (29, 57): 2,
    (57, 92): 3,
    (92, 120): 4,
    (120, 148): 5,
    (148, 183): 6,
    (183, 211): 7,
    (211, 239): 8,
    (239, 274): 9,
    (274, 302): 10,
    (302, 330): 11,
    (330, 365): 12,      
    })

In [33]:
s_year_rkd_monthOfYear = RangeKeyDict({
    (1, 29): 1,  ### All keys in the 2nd position are 1 greater than the actual day count because the range function is n + 1
    (29, 57): 2,
    (57, 92): 3,
    (92, 120): 4,
    (120, 148): 5,
    (148, 183): 6,
    (183, 218): 7,
    (218, 246): 8,
    (246, 281): 9,
    (281, 309): 10,
    (309, 337): 11,
    (337, 372): 12,      
    })

In [34]:
def get_monthOfFiscalYear(row):
    if row.fiscalyear in special_years:
        return s_year_rkd_monthOfYear[row.daynumber]
    else:
        return n_year_rkd_monthOfYear[row.daynumber]  

In [35]:
test['month_of_fiscalyear'] = test.apply(get_monthOfFiscalYear, axis=1)

In [36]:
test.loc[265:275]

,date,fiscalyear,daynumber,fiscalquarter,days_in_quarter,quarterdays_end,quarterdays_begin,month_of_quarter,month_of_fiscalyear
268,1999-10-07,2000,68,1,91,91,0.0,3,3
272,1999-10-08,2000,69,1,91,91,0.0,3,3


In [37]:
n_year_rkd_weekOfYear = RangeKeyDict({
    (1, 8): 1,      
    (8, 15): 2,
    (15, 22): 3,    
    (22, 29): 4,
    (29, 36): 5,    
    (35, 43): 6,
    (43, 50): 7,    
    (50, 57): 8,
    (57, 64): 9,    
    (64, 71): 10,
    (71, 78): 11,   
    (78, 85): 12,
    (85, 92): 13,   
    (92, 99): 14,
    (99, 106): 15,  
    (106, 113): 16,
    (113, 120): 17, 
    (120, 127): 18,
    (127, 134): 19, 
    (134, 141): 20,
    (141, 148): 21, 
    (148, 155): 22,
    (155, 162): 23, 
    (162, 169): 24,
    (169, 176): 25, 
    (176, 183): 26,
    (183, 190): 27, 
    (190, 197): 28,
    (197, 204): 29, 
    (204, 211): 30,
    (211, 218): 31, 
    (218, 225): 32,
    (225, 232): 33, 
    (232, 239): 34,
    (239, 246): 35, 
    (246, 253): 36,
    (253, 260): 37, 
    (260, 267): 38,
    (267, 274): 39, 
    (274, 281): 40,
    (281, 288): 41, 
    (288, 295): 42,
    (295, 302): 43, 
    (302, 309): 44,
    (309, 316): 45, 
    (316, 323): 46,
    (323, 330): 47, 
    (330, 337): 48,
    (337, 344): 49, 
    (344, 351): 50,
    (351, 358): 51, 
    (358, 365): 52   
    })

In [38]:
s_year_rkd_weekOfYear = RangeKeyDict({
    (1, 8): 1,      
    (8, 15): 2,
    (15, 22): 3,    
    (22, 29): 4,
    (29, 36): 5,    
    (35, 43): 6,
    (43, 50): 7,    
    (50, 57): 8,
    (57, 64): 9,    
    (64, 71): 10,
    (71, 78): 11,   
    (78, 85): 12,
    (85, 92): 13,   
    (92, 99): 14,
    (99, 106): 15,  
    (106, 113): 16,
    (113, 120): 17, 
    (120, 127): 18,
    (127, 134): 19, 
    (134, 141): 20,
    (141, 148): 21, 
    (148, 155): 22,
    (155, 162): 23, 
    (162, 169): 24,
    (169, 176): 25, 
    (176, 183): 26,
    (183, 190): 27, 
    (190, 197): 28,
    (197, 204): 29, 
    (204, 211): 30,
    (211, 218): 31, 
    (218, 225): 32,
    (225, 232): 33, 
    (232, 239): 34,
    (239, 246): 35, 
    (246, 253): 36,
    (253, 260): 37, 
    (260, 267): 38,
    (267, 274): 39, 
    (274, 281): 40,
    (281, 288): 41, 
    (288, 295): 42,
    (295, 302): 43, 
    (302, 309): 44,
    (309, 316): 45, 
    (316, 323): 46,
    (323, 330): 47, 
    (330, 337): 48,
    (337, 344): 49, 
    (344, 351): 50,
    (351, 358): 51, 
    (358, 365): 52,
    (365, 372): 53,
    })

In [39]:
def get_weekOfFiscalYear(row):
    if row.fiscalyear in special_years:
        return s_year_rkd_weekOfYear[row.daynumber]
    else:
        return n_year_rkd_weekOfYear[row.daynumber]

In [40]:
test['week_of_fiscalyear'] = test.apply(get_weekOfFiscalYear, axis=1)

In [41]:
n_year_rkd_weekOfQtr = RangeKeyDict({
    (1, 8): 1,      
    (8, 15): 2,
    (15, 22): 3,    
    (22, 29): 4,
    (29, 36): 5,    
    (35, 43): 6,
    (43, 50): 7,    
    (50, 57): 8,
    (57, 64): 9,    
    (64, 71): 10,
    (71, 78): 11,   
    (78, 85): 12,
    (85, 92): 13,   
    (92, 99): 1,
    (99, 106): 2,  
    (106, 113): 3,
    (113, 120): 4, 
    (120, 127): 5,
    (127, 134): 6, 
    (134, 141): 7,
    (141, 148): 8, 
    (148, 155): 9,
    (155, 162): 10, 
    (162, 169): 11,
    (169, 176): 12, 
    (176, 183): 13,
    (183, 190): 1, 
    (190, 197): 2,
    (197, 204): 3, 
    (204, 211): 4,
    (211, 218): 5, 
    (218, 225): 6,
    (225, 232): 7, 
    (232, 239): 8,
    (239, 246): 9, 
    (246, 253): 10,
    (253, 260): 11, 
    (260, 267): 12,
    (267, 274): 13, 
    (274, 281): 1,
    (281, 288): 2, 
    (288, 295): 3,
    (295, 302): 4, 
    (302, 309): 5,
    (309, 316): 6, 
    (316, 323): 7,
    (323, 330): 8, 
    (330, 337): 9,
    (337, 344): 10, 
    (344, 351): 11,
    (351, 358): 12, 
    (358, 365): 13   
    })

In [42]:
s_year_rkd_weekOfQtr = RangeKeyDict({
    (1, 8): 1,      
    (8, 15): 2,
    (15, 22): 3,    
    (22, 29): 4,
    (29, 36): 5,    
    (35, 43): 6,
    (43, 50): 7,    
    (50, 57): 8,
    (57, 64): 9,    
    (64, 71): 10,
    (71, 78): 11,   
    (78, 85): 12,
    (85, 92): 13,   
    (92, 99): 1,
    (99, 106): 2,  
    (106, 113): 3,
    (113, 120): 4, 
    (120, 127): 5,
    (127, 134): 6, 
    (134, 141): 7,
    (141, 148): 8, 
    (148, 155): 9,
    (155, 162): 10, 
    (162, 169): 11,
    (169, 176): 12, 
    (176, 183): 13,
    (183, 190): 1, 
    (190, 197): 2,
    (197, 204): 3, 
    (204, 211): 4,
    (211, 218): 5, 
    (218, 225): 6,
    (225, 232): 7, 
    (232, 239): 8,
    (239, 246): 9, 
    (246, 253): 10,
    (253, 260): 11, 
    (260, 267): 12,
    (267, 274): 13, 
    (274, 281): 1,
    (281, 288): 2, 
    (288, 295): 3,
    (295, 302): 4, 
    (302, 309): 5,
    (309, 316): 6, 
    (316, 323): 7,
    (323, 330): 8, 
    (330, 337): 9,
    (337, 344): 10, 
    (344, 351): 11,
    (351, 358): 12, 
    (358, 365): 13,
    (365, 372): 14,
    })

In [43]:
# get the week of the fiscal quarter
def get_weekOfFiscalQuarter(row):
    if row.fiscalyear in special_years:
        return s_year_rkd_weekOfQtr[row.daynumber]
    else:
        return n_year_rkd_weekOfQtr[row.daynumber]

In [44]:
test['week_of_fiscalquarter'] = test.apply(get_weekOfFiscalQuarter, axis=1)

In [45]:
input_a_date = '2/29/2096'
thisdate = dt.datetime.strptime(input_a_date, '%m/%d/%Y')
thisdate = thisdate.isoformat()[:10]
df_maxdate = test.date.max()
df_maxdate

'2024-07-27'

In [46]:
def get_fiscal_year(input_a_date):
    thisdate = dt.datetime.strptime(input_a_date, '%m/%d/%Y')
    thisdate = thisdate.isoformat()[:10]
    df_maxdate = test.date.max()
    if thisdate > df_maxdate:
        print(f"The date queried is greater than the current maximum date: {df_maxdate}. Please input an earlier date.")
    else:
        try:
            checkyear = test.loc[test.date == thisdate]['fiscalyear'].values[0]
            return checkyear
        except ValueError:
            print(f"The date provided does not exist; Please check your input and re-enter a valid date.")
        

In [47]:
get_fiscal_year('2/28/2023')

2023

In [48]:
# input_a_date = '5/26/2028'
# date = dt.datetime.strptime(input_a_date, '%m/%d/%Y')

## Add CALENDAR MONTH NUMBER and CALENDAR DAY OF MONTH to the dataframe

In [49]:
test.loc[test['date']=='2104-02-29']

,date,fiscalyear,daynumber,fiscalquarter,days_in_quarter,quarterdays_end,quarterdays_begin,month_of_quarter,month_of_fiscalyear,week_of_fiscalyear,week_of_fiscalquarter


In [50]:
endtime=dt.datetime.now()
elapsed = endtime - starttime
elapsed

datetime.timedelta(seconds=174, microseconds=758015)

In [51]:
test.shape

(9128, 11)

In [52]:
test

,date,fiscalyear,daynumber,fiscalquarter,days_in_quarter,quarterdays_end,quarterdays_begin,month_of_quarter,month_of_fiscalyear,week_of_fiscalyear,week_of_fiscalquarter
0,1999-08-01,2000,1,1,91,91,0.0,1,1,1,1
4,1999-08-02,2000,2,1,91,91,0.0,1,1,1,1
8,1999-08-03,2000,3,1,91,91,0.0,1,1,1,1
12,1999-08-04,2000,4,1,91,91,0.0,1,1,1,1
16,1999-08-05,2000,5,1,91,91,0.0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
36495,2024-07-23,2024,360,4,91,364,273.0,3,12,52,13
36499,2024-07-24,2024,361,4,91,364,273.0,3,12,52,13
36503,2024-07-25,2024,362,4,91,364,273.0,3,12,52,13
36507,2024-07-26,2024,363,4,91,364,273.0,3,12,52,13


In [57]:
FY = dict(zip(test.date, test.fiscalyear))
FQ = dict(zip(test.date, test.fiscalquarter))


In [59]:
FQ.get('2021-03-05')

3